# Make a model & Run it with OpenSEES




In [117]:
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)

import numpy as np
import functools
import pprint
pp = pprint.PrettyPrinter(indent=4)


import sys
sys.path.append(r'C:\Users\tgolecki\AppData\Local\Continuum\anaconda3\Lib\site-packages\openseespy')
#from opensees import *
import opensees

In [118]:
#consistent units: in,lb

#Define Mesh Parameters
#dx,dy

#Define Deck Geometry
#dx,dy

#Define Girder Parameters
#y,prop

#Define Diaphragm Parameters
#x,prop



In [119]:
#Make a dictionary of nodes, mesh is 27x79 
#nodes on deck
NODES={}
for j in range(1,28):
    for i in range(1,80):
        n=100*j+i
        x=(i-1)*10
        y=(j-14)*11
        z=0
        NODES[n]=[x,y,z]

#nodes on girders
for j in (2,10,18,26):
    for i in range(1,80):
        n=10000+100*j+i
        x=(i-1)*10
        y=(j-14)*11
        z=-24
        NODES[n]=[x,y,z]
        
#nodes on BC
for j in (2,10,18,26):
    for i in (1,79):
        n=20000+100*j+i
        x=(i-1)*10
        y=(j-14)*11
        z=-48
        NODES[n]=[x,y,z]

In [120]:
#MATERIALS
MATERIALS={}
#MATERIALS
#steel
MATERIALS[1] = {"E":29000000.0,"Nu":0.29,"Rho":490/12**3}
#conc
MATERIALS[2] = {"E":57000*(3000)**0.5,"Nu":0.2,"Rho":150/12**3}

for m in MATERIALS:
    MATERIALS[m]['G'] = MATERIALS[m]['E']/(2*(1+MATERIALS[m]['Nu']))

In [121]:
#BEAM SECTIONS
SECTIONS={}
#SECTIONS[i] = ["A","Ix","Iy","J","Ax","Ay"]
SECTIONS[1] = {"A":12.475,  "Ix": 549.035, "Iy": 15.6833, "J": 1.10832, "Ax": 1.66727,"Ay": 7.40825}
SECTIONS[2] = {"A":37.8712, "Ix": 6607.75, "Iy": 217.235, "J": 6.73246, "Ax": 16.6574,"Ay": 18.2646}
SECTIONS[3] = {"A":45.6822, "Ix": 8414.14, "Iy": 288.743, "J": 18.3722, "Ax": 23.2446,"Ay": 18.6911}
SECTIONS[4] = {"A":44.5148, "Ix": 8277.46, "Iy": 276.798, "J": 15.5185, "Ax": 22.126 ,"Ay": 18.7483}

#DECK PROPERTIES
DECK={}
DECK[1]={"thick":7.25,"nsm":0,"mat":1}

In [122]:
#SHELL ELEMENTS FOR DECK
SHELLS={}
mat=1
prop=1
for j in range(1,27):
    for i in range(1,79):
        n1=100*j+i
        n2=100*(j+1)+i
        n3=100*(j+1)+(i+1)
        n4=100*j+(i+1)
        SHELLS[n1]=[[n1,n2,n3,n4],prop]



In [123]:
#BEAM ELEMENTS FOR GIRDERS
BEAMS={}
#BEAMS[i]=[n1,n2,mat,prop]

mat=1
prop=1
for j in (2,10,18,26):
    for i in range(1,79):
        n1=10000+100*j+i
        n2=10000+100*j+i+1
        if j in [2,26] and i in range(16,65):
            prop=3
        elif j in [10,18] and i in range(17,64):
            prop=4
        else:
            prop=2
        BEAMS[n1]=[n1,n2,mat,prop]
        
#BEAM ELEMENTS FOR DIAPHRAGMS
for j in (2,10,18):
    for i in (1,20,40,60,79):
        n1=10000+100*j+i
        n2=10000+100*(j+8)+i
        BEAMS[n1+10000]=[n1,n2,1,1]


In [124]:
#LINKS BETWEEN GIRDERS AND DECK
LINKS={}
for j in (2,10,18,26):
    for i in range(1,80):
        n2=100*j+i
        n1=10000+100*j+i
        LINKS[n1]=[n1,n2]
        #BEAMS[n1]=[n1,n2,mat,prop]

#BOUNDARY CONDITON LINKS
for j in (2,10,18,26):
    for i in (1,79):
        n1=20000+100*j+i
        n2=10000+100*j+i
        LINKS[n1]=[n1,n2]
        #BEAMS[n1]=[n1,n2,mat,prop]


In [125]:
#BOUNDARY CONDITIONS
#6 dof per node, 1=restrained, 0=free
BC={}
for j in (2,10,18,26):
    for i in (1,79):
        n=20000+100*j+i
        if i==1:
            BC[n]=[1,1,1,0,0,0]
        else:
            BC[n]=[0,0,1,0,0,0]

In [126]:
#build a plot
#nodes
x=[NODES[n][0] for n in sorted(NODES.keys())]
y=[NODES[n][1] for n in sorted(NODES.keys())]
z=[NODES[n][2] for n in sorted(NODES.keys())]
node_trace = go.Scatter3d(x=x,y=y,z=z, mode='markers',marker=dict(size=1,color='blue'),name='Nodes')

#beams
DEFAULT_PLOTLY_COLORS=['rgb(31, 119, 180)', 'rgb(255, 127, 14)',
                       'rgb(44, 160, 44)', 'rgb(214, 39, 40)',
                       'rgb(148, 103, 189)', 'rgb(140, 86, 75)',
                       'rgb(227, 119, 194)', 'rgb(127, 127, 127)',
                       'rgb(188, 189, 34)', 'rgb(23, 190, 207)']

beam_trace=[]
for b in sorted(BEAMS.keys()):
    if b==sorted(BEAMS.keys())[0]:
        showlegend=True
    else:
        showlegend=False
    beam_trace.append(go.Scatter3d(x=[NODES[BEAMS[b][0]][0],NODES[BEAMS[b][1]][0]],
                                   y=[NODES[BEAMS[b][0]][1],NODES[BEAMS[b][1]][1]],
                                   z=[NODES[BEAMS[b][0]][2],NODES[BEAMS[b][1]][2]],
                                   mode='lines',legendgroup='beams',showlegend=showlegend,
                                   name='Beams',line=dict(color=DEFAULT_PLOTLY_COLORS[BEAMS[b][3]])))
#shells (as a mesh)
shell_trace=[]
faces=[]
shellnodes={}
counter=0
x=[]
y=[]
z=[]
for s in sorted(SHELLS.keys()):
    for n in SHELLS[s][0]:
        x.append(NODES[n][0])
        y.append(NODES[n][1])
        z.append(NODES[n][2])
        shellnodes[n]=counter
        counter+=1
faces=[]
for s in sorted(SHELLS.keys()):
    faces.append(np.asarray([shellnodes[SHELLS[s][0][0]],shellnodes[SHELLS[s][0][1]],shellnodes[SHELLS[s][0][2]]]))
    faces.append(np.asarray([shellnodes[SHELLS[s][0][2]],shellnodes[SHELLS[s][0][3]],shellnodes[SHELLS[s][0][0]]]))
    
points3D=np.vstack((x,y,z)).T
tri_vertices=list(map(lambda index: points3D[index], faces))
I,J,K=([triplet[c] for triplet in faces] for c in range(3))
shell_trace=dict(type='mesh3d',lighting=dict(fresnel=0.2,roughness=0.5,specular=0.05,ambient=0.8,diffuse=0.8),
               flatshading=True,opacity=0.25,x=x,y=y,z=z,color = 'green', i=I,j=J,k=K,name='Shells',legendgroup='shells')
lists_coord=[[[T[k][c] for k in range(3)]+[None] for T in tri_vertices] for c in range(3)]

Xe, Ye, Ze=[functools.reduce(lambda x,y: x+y, lists_coord[k]) for k in range(3)]
shelledge_trace=dict(type='scatter3d',legendgroup='shells',showlegend=False,x=Xe,y=Ye,z=Ze,
           mode='lines',line=dict(color= 'rgb(150,150,150)',width=1.0))

#links
link_trace=[]
for l in sorted(LINKS.keys()):
    if l==sorted(LINKS.keys())[0]:
        showlegend=True
    else:
        showlegend=False   
    link_trace.append(go.Scatter3d(x=[NODES[LINKS[l][0]][0],NODES[LINKS[l][1]][0]],
                                   y=[NODES[LINKS[l][0]][1],NODES[LINKS[l][1]][1]],
                                   z=[NODES[LINKS[l][0]][2],NODES[LINKS[l][1]][2]],
                                   mode='lines',legendgroup='links',showlegend=showlegend,name='Links',line=dict(color='black')))
#layout
noaxis=dict(showbackground=False,showline=False,zeroline=False,showgrid=False,showticklabels=False,title='')
layout=dict(title='Model',scene=dict(aspectmode='data',xaxis=noaxis,yaxis=noaxis,zaxis=noaxis))
fig = go.Figure(data=[node_trace]+beam_trace+link_trace+[shell_trace]+[shelledge_trace],layout=layout)
print("Ready")

Ready


In [127]:
#plot the model
py.iplot(fig)

In [128]:
# ------------------------------
# Start of model generation
# -----------------------------

# remove existing model
opensees.wipe()

# set modelbuilder, basic 3d model
opensees.model('basic', '-ndm', 3)
#---------------------------------------------------------------------------------------
# create nodes

for n in sorted(NODES.keys()):
    opensees.node(int(n), *[float(NODES[n][0]), float(NODES[n][1]),float(NODES[n][2])])

#---------------------------------------------------------------------------------------
# boundary conditions

for n in sorted(BC.keys()):
    opensees.fix(int(n),*BC[n])

#---------------------------------------------------------------------------------------
# define materials

for m in MATERIALS:
    opensees.nDMaterial('ElasticIsotropic', int(m), float(MATERIALS[m]['E']),
                        float(MATERIALS[m]['Nu']), float(MATERIALS[m]['Rho']))

#---------------------------------------------------------------------------------------    
# define beam elements 

transfTag=1
#transfArgs=[$vecxzX,$vecxzY,$vecxzZ]
#no transformation 
transfArgs=[1.0,0.0,0.0]
opensees.geomTransf("Linear", int(transfTag),*transfArgs)

    
for b in BEAMS:
    mat=BEAMS[b][2]
    prop=BEAMS[b][3]
    opensees.element('ElasticTimoshenkoBeam', int(b), *[BEAMS[b][0],BEAMS[b][1]],
                     MATERIALS[mat]['E'], MATERIALS[mat]['G'], SECTIONS[prop]['A'], 
                     SECTIONS[prop]['J'], SECTIONS[prop]['Iy'],SECTIONS[prop]['Ix'], SECTIONS[prop]['Ay'],
                     SECTIONS[prop]['Ax'],transfTag,float(SECTIONS[prop]['A']*MATERIALS[mat]['Rho']),"-cMass")
    
#---------------------------------------------------------------------------------------
# define shell sections
for d in sorted(DECK.keys()):
    mat = DECK[d]['mat']
    opensees.section('ElasticMembranePlateSection', int(d), float(MATERIALS[mat]['E']),
            float(MATERIALS[mat]['Nu']+DECK[d]['nsm']/DECK[d]['thick']),
            float(DECK[d]['thick']), float(MATERIALS[mat]['Rho']))

# define shell elements as ShellMITC4 elements
for s in sorted(SHELLS.keys()): 
    opensees.element('ShellMITC4',int(s),*SHELLS[s][0],int(SHELLS[s][1]))
    
#---------------------------------------------------------------------------------------    
#define links    
for l in sorted(LINKS.keys()):
    opensees.rigidLink('beam', LINKS[l][0], LINKS[l][1])

#---------------------------------------------------------------------------------------
# define loads
# create TimeSeries
opensees.timeSeries("Constant", 1)

# create a plain load pattern
opensees.pattern("Plain", 1, 1)

# unit point load at node 240 in -z direction
opensees.load(int(240), *[0.0,0.0,-1.0,0.0,0.0,0.0])



In [129]:
# ------------------------------
# Start of analysis generation
# ------------------------------
# create SOE
opensees.system("BandSPD")

# create DOF number
opensees.numberer("RCM")

# create constraint handler
#opensees.constraints("Penalty",1.0,1.0)
#opensees.constraints("Lagrange",1.0,1.0)
opensees.constraints("Transformation")

# create integrator
opensees.integrator("LoadControl", 1.0)

# create algorithm
opensees.algorithm("Linear")

# create analysis object
opensees.analysis("Static")

In [130]:
# perform the analysis
opensees.analyze(1)

-3